In [1]:
!pip install kaggle

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.4 MB 2.9 MB/s eta 0:00:01
   ------------------------------------ --- 1.3/1.4 MB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 2.4 MB/s  0:00:00
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   --- ------------------------------------ 0.8/10.1 MB 6.1 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/10.1 MB 5.9 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/10.1 MB 3.5 MB/s eta 0:00:03
   ------------- -------------------------- 3.4/10.1 MB 4.3 MB/s eta 0:00:02
   ------------------ --------------------- 4.7/10.1 MB 4.6 MB/s eta 0:00:02
   -------------------- ------------------- 5.2/10.1 MB 4.2 MB/s eta 0:00:02
   ----------------------- ---------------- 6.0/10.1 MB 4.2 MB/s eta 0:00:01
   -------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import kaggle
kaggle.api.authenticate()
# Replace 'owner/dataset-name' with your actual dataset path from the URL
kaggle.api.dataset_download_files('Gabriel Preda/Nigerian Presidential Election 2023 Tweets', path='./', unzip=True)

Could not find kaggle.json. Make sure it's located in C:\Users\DR AKIN O\.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


NameError: name 'exit' is not defined